<a href="https://colab.research.google.com/github/shanu-gandhi-iiith/WikiSearchEngine/blob/master/phase2_indexer_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/My\ Drive/wikiproj


Mounted at /content/drive
/content/drive/My Drive/wikiproj


In [ ]:
#########################
mapparid_id={} ###map of title vs id
res=0 ############### make unique id
mapstem={} ##### stm wrd map
#### id to title mapping
!ls uz
totaltok=0

enwiki-20200801-pages-articles-multistream1.xml-p1p30303      index
enwiki-20200801-pages-articles-multistream2.xml-p30304p88444  short.xml


In [ ]:
import re
import string
import sys 
import time
import os
import pickle


path = "uz/enwiki-20200801-pages-articles-multistream2.xml-p30304p88444"
indexpath = "uz/index"
stats = "uz/stats.txt"

os.system("mkdir -p "+indexpath)

##########################
import nltk
import re
nltk.download('punkt',quiet=True)
nltk.download('stopwords',quiet=True)
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem import SnowballStemmer 
from nltk.tokenize import word_tokenize 

# import spacy

custset={" ","a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could","redirect","couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "wiktionary", "cite","reflist"}
# nlp = spacy.load('en')
custset=custset.union(set(stopwords.words('english')))
# custset=custset.union(set(spacy.lang.en.stop_words.STOP_WORDS))
ps =nltk.stem.SnowballStemmer('english')




reg1=re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',re.DOTALL)
reg2=re.compile(r'{{v?cite(.*?)}}',re.DOTALL)
reg3=re.compile(r'{\|(.*?)\|}',re.DOTALL)

# reg4=re.compile(r"[~`!@#$%-^*+{\[}\]\|\\<>/?]",re.DOTALL)


def rem_pun(line):
    return line.translate(str.maketrans('','',string.punctuation))


# re.split(r'[^A-Za-z0-9]+', text)
def hasNumbers(instr):
    return any(char.isdigit() for char in instr)

def finalstr(body):
  global mapstem,totaltok
  body = body.lower()
  body = reg1.sub(' ', body)
  body = reg2.sub(' ', body)
  body = reg3.sub(' ', body)
  body=body.replace('"', ' ')
  body=body.replace("'", ' ')
  
  # body = reg4.sub(' ', body)
  body = rem_pun(body)


  lis2=body.split()


  # lis2=re.split(r'[^A-Za-z0-9]+', body)
  stmd=""
  for i in lis2:
    totaltok=totaltok+1
    if (not i in custset) and len(i) > 2 and len(i)<=30:
      i=i.replace('"', '')
      i=i.replace("'", '')
      if i in mapstem:
        stmd+=mapstem[i]+" "
      else:
        x=ps.stem(i)
        mapstem[i]=x;
        stmd+=x+" "
  return (stmd)



# map making final

import re
import xml.sax

mapgl={}

titlestr="" 
infoboxstr=""
bodystr=""
categorystr="" 
extlinkstr=""
referencestr=""
cpid=-1
titleg=""


def parsetxt_i_b_c(curline):
  curline=curline.lower()
  ib=""
  bd=""
  ct=""
  et=""
  rf=""
  lisw=curline.split()
  cflg=0
  iflg=0
  bal=0
  for wrd in lisw:
    if wrd.startswith("[[category"): # categs
      cflg=1
      iflg=0
      eflg=0
      rflg=0
    
    elif cflg==1 and wrd.startswith("]]"): ## cat end
      cflg=0;
      eflg=0
      rflg=0
    
    elif iflg==0 and wrd.startswith("{{infobox"): ## info start
      iflg=1
      bal=0
      cflg=0
      eflg=0
      rflg=0    

    if iflg==1:
      if wrd.find("{{"):
        bal=bal+1;
      if wrd.find("}}"):
        bal=bal-1;

    if bal==0 :
      iflg=0
      bal=0
        
    if cflg==1:
      ct+=wrd[11:]+" "

    elif iflg==1:
      ib+=wrd+" "
    
    elif cflg==0 and iflg==0:
      bd+=wrd+" "
  
  rs=bd.find("==references")
  if rs==-1 :
    rs=bd.find("== references")

  if  rs != -1 :
    re=bd.find("==",rs+18)
    if re==-1:
      rf+=bd[rs+16:]
      bd=bd[0:rs-1]
    else:
      rf+=bd[rs+16:re]
      bd=bd[0:rs-1] + bd[re+1: ]


  es=bd.find("== external links")
  if es==-1:
    es=bd.find("==external links")
  if es==-1:
    es=bd.find("=external links")
  if es==-1:
    es=bd.find("= external links")

  if es != -1 :
    ee1=bd.find("==", es+22)
    ee2=bd.find("[[ca", es+22)
    if ee1==-1 and ee2 ==-1:
      et+=bd[es+20:]
      bd=bd[0:es]
    elif ee1==-1 and ee2 !=-1:
      et+=bd[es+20:ee2]
      bd=bd[0:es]+bd[ee2:]
    elif ee1!=-1 and ee2==-1:
      et+=bd[es+20:ee1]
      bd=bd[0:es]+bd[ee1:]
    else:
      mn=min(ee1, ee2)
      et+=bd[es+20:mn]
      bd=bd[0:es]+bd[mn:]

  return ib, bd, ct,et,rf

class XMLHandler(xml.sax.ContentHandler):
    def __init__(self):
        self.CurrentData=""
        self.model=""
        self.speed=""
        self.title=""
        self.text=""
        self.parentid=-1
    
    def startElement(self, tag, attributes):
        self.CurrentData = tag
        global res;
        if tag == "title":
            print(res)
            res=res+1;
            if (res %100) == 0 :
              print(res,": %s seconds " % (time.time() - start_time))
        
    def endElement(self, tag):
        global cpid,titleg;
        if (self.CurrentData =="title"):
            # print("PID T",cpid)
            # print("Title:",self.title)
            self.title=""
        
        elif (self.CurrentData =="parentid"):
            # print("Pid:",self.parentid)
            self.parentid=""
        elif (self.CurrentData =="text"):
            # print("Text",self.text)
            global infoboxstr, bodystr, categorystr, extlinkstr, referencestr,res
            counttit,counti,countc,countb,counte,countr=0,0,0,0,0,0,
            # parse items from text
            infoboxstr, bodystr, categorystr, extlinkstr, referencestr=parsetxt_i_b_c(self.text)
            ###no print
            # print(finalstr(bodystr))     
            titlestr=finalstr(titleg)
            infoboxstr=finalstr(infoboxstr)
            categorystr=finalstr(categorystr)
            extlinkstr=finalstr(extlinkstr)
            referencestr=finalstr(referencestr)
            bodystr=finalstr(bodystr)
            mapdoc={} #map< string ,lis[0,1,2,3,4,5]> lis[title,info,categ,ext,ref,body]
            
            #############enter in common doc map 0-title 1- info 2-categ 3-ext 4-ref 5-body
            
            listit=titlestr.split(' ')
            counttit+=len(listit)
            for i in listit:
              if i in mapdoc:
                v1=mapdoc[i]
                v1[0]=v1[0]+1
                mapdoc[i]=v1
              else:
                mapdoc[i]=[1,0,0,0,0,0]
            
            listit=infoboxstr.split(' ')
            counti+=len(listit)
            for i in listit:
              if i in mapdoc:
                v1=mapdoc[i]
                v1[1]=v1[1]+1
                mapdoc[i]=v1
              else:
                mapdoc[i]=[0,1,0,0,0,0]
            
            listit=categorystr.split(' ')
            countc+=len(listit)
            for i in listit:
              if i in mapdoc:
                v1=mapdoc[i]
                v1[2]=v1[2]+1
                mapdoc[i]=v1
              else:
                mapdoc[i]=[0,0,1,0,0,0]
            
            listit=extlinkstr.split(' ')
            counte+=len(listit)
            for i in listit:
              if i in mapdoc:
                v1=mapdoc[i]
                v1[3]=v1[3]+1
                mapdoc[i]=v1
              else:
                mapdoc[i]=[0,0,0,1,0,0]
            
            listit=referencestr.split(' ')
            countr+=len(listit)
            for i in listit:
              if i in mapdoc:
                v1=mapdoc[i]
                v1[4]=v1[4]+1
                mapdoc[i]=v1
              else:
                mapdoc[i]=[0,0,0,0,1,0]
            
            listit=bodystr.split(' ')
            countb+=len(listit)
            for i in listit:
              if i in mapdoc:
                v1=mapdoc[i]
                v1[5]=v1[5]+1
                mapdoc[i]=v1
              else:
                mapdoc[i]=[0,0,0,0,0,1]
            
            # mapgl
            ##############            
            ###enter in global map
            for i in mapdoc:
              curlis=mapdoc[i]
              # curlis.append(cpid)
              mapparid_id[res]=[titleg,counttit,counti,countc,countb,counte,countr,(counttit+counti+countc+countb+counte+countr)]
              curlis.append(res)
              if i in mapgl:
                cl=mapgl[i]
                cl.append(curlis)
                mapgl[i]=cl
              else:
                mapgl[i]=[curlis]
            

            
            #########
            infoboxstr=""
            bodystr=""
            categorystr=""
            extlinkstr=""
            referencestr=""
            
            self.text=""
        self.CurrentData=""
    
    def characters(self, content):
        global cpid,titleg
        if(self.CurrentData=="title"):
            self.title+=content
            titleg=content
        elif(self.CurrentData=="text"):
            self.text+=content
        elif(self.CurrentData=="parentid"):
            self.parentid=content
            cpid=content

start_time = time.time()

parser=xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_namespaces,0)
Handler=XMLHandler()
parser.setContentHandler(Handler)
# parser.parse(pathsc)
parser.parse(path)
print("done")
print("--- %s seconds ---" % (time.time() - start_time))


print(len(mapgl))

start_time=time.time()


mapgl3={}

for key in (mapgl):
  i2=key
  i2=i2.replace("'","")
  i2=i2.replace('"',"")
  i2=i2.replace('-',"")
  # i2=re.sub(r'[^A-Za-z0-9 ]+', '', i2)
  if i2 =="":
    continue
  mapgl3[i2]=mapgl[key]

mapgl=mapgl3
mapgl2={}
for key in sorted(mapgl):
    mapgl2[key]=mapgl[key]

print("writing index file")

with open("indexsf/ind2", 'w') as writefile:
  for i in mapgl2:
    if i =="":
      continue
    writefile.write(i+"=")
    for j in mapgl2[i]:
      ind=-1
      for k in j:
        ind=ind+1
        if k==0:
          continue #map< string ,lis[0,1,2,3,4,5]> lis[title,info,categ,ext,ref,body,docid()]
        if ind==0:
          writefile.write("t"+str(k)+"-")
        if ind==1:
          writefile.write("i"+str(k)+"-")
        if ind==2:
          writefile.write("c"+str(k)+"-")
        if ind==3:
          writefile.write("e"+str(k)+"-")
        if ind==4:
          writefile.write("r"+str(k)+"-")
        if ind==5:
          writefile.write("b"+str(k)+"-")
        if ind==6:
          writefile.write("d"+str(k)+":")
    writefile.write("\n")
print("--- %s seconds ---" % (time.time() - start_time))


# with open("indexsf/globmap2", 'w') as writefile:
#   for i in mapparid_id:
#     str2=""
#     for k in mapparid_id[i]:
#       str2+=str(k)+":"
#     # print(str(i)+"="+str2+"\n")
#     writefile.write(str(i)+"="+str2+"\n")

with open("indexsf/globmap", 'a+') as writefile:
  for i in mapparid_id:
    str2=""
    for k in mapparid_id[i]:
      str2+=str(k)+":"
    # print(str(i)+"="+str2+"\n")
    writefile.write(str(i)+"="+str2+"\n")



print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
!tail -10 "indexsf/ind2"
# j=0
# for i in mapgl2:
#   j=j+1
#   if i=="":
#     continue
#   if j==10:
#     break
#   print(i,mapgl2[i])

𓈗𓇋𓃀𓅱𓈉=i1-d47602:
𓍁𓈖𓏭𓆱=i1-d31225:
𖬌𖬣𖬵=b1-d38953:
𝐍𝐨sup𝕯sup=b2-d34021:
𝔽sub2sublinear=b1-d30438:
𝕋sub𝕟sub=b1-d26643:
𝕕big=b1-d26643:
𞄀𞄩𞄰=b1-d38953:
🧫span=b1-d29373:
𢒉ref=b1-d36315:


In [ ]:
def merge2f(file1, file2, file3):
  import os
  #actual merge 2 files
  # !head -10 "drive/My Drive/IRE/phase2/textindex2.txt"
  file1 = open(file1, 'r') 
  file2 = open(file2, 'r') 
  file3 = open(file3, 'w+') 
  count=0
  line1 = file1.readline() 
  line2 = file2.readline() 
  while line1 or line2:
    if line1 and not line2:
      file3.write(line1)
      line1=file1.readline()
    elif line2 and not line1:
      file3.write(line2)
      line2=file2.readline()
    else:
      line10=line1.split("=")[0]
      line20=line2.split("=")[0]
      if (line10 < line20):
        file3.write(line1)
        line1=file1.readline()
      elif (line10 > line20):
        file3.write(line2)
        line2=file2.readline()
      else:
        linecommon=""
        linecommon=linecommon+line10+"="
        res=""
        res =res+line1.split("=")[1][:-1]
        res =res+line2.split("=")[1]
        
        linecommon=linecommon+res
        file3.write(linecommon)
        line1=file1.readline()
        line2=file2.readline()
  file1.close() 
  file2.close() 
  file3.close() 

In [ ]:
p1="indexsf/ind1"
p2="indexsf/ind2"
p3="indexsf/indmerged"
merge2f(p1,p2,p3)


In [ ]:
##############convert globalmap to map
# !head -10 indexsf/globmap
map_docidattr={}
f1=open("indexsf/globmap","r")
# line= 58576=Phosphor:2:1:6:2622:38:2:2671:
while True:
  line=f1.readline()
  if not line:
    break
  line=line.strip()
  did=line.split("=")[0]
  attr=line.split("=")[1]
  map_docidattr[did]=attr

pickle_out = open("indexsf/dict.pickle","wb")
pickle.dump(map_docidattr, pickle_out)
pickle_out.close()

In [ ]:
# !head -10 indexsf/indmerged
# !wc -l indexsf/indmerged
os.system("mkdir -p indexsf/chunksdir")

0

In [ ]:
# !head  indexs/chunksf/secndind
# !tail -10 indexsf/indmerged

In [ ]:
#################split index and create small files

indf=open("indexsf/chunksdir/secndind","w")

readf=open("indexsf/indmerged","r")
line = readf.readline() 
# j=-1
j=-1
writefile=open("indexsf/chunksdir/chunk_0","w")
indf.write("chunk_0:"+str(line.split("=")[0])+"\n")
while True:
	j=j+1
	# if j==30003:
	# 	break
	if not line: 
		break
	if j%1000000==0 and j!=0:
		indf.write("chunk_"+str(int(j/1000000))+":"+str(line.split("=")[0])+"\n")
		writefile.close()
		writefile=open("indexsf/chunksdir/chunk_"+str(int(j/1000000)), "w")

	writefile.write(line)
	line = readf.readline() 
	if not line: 
		break
writefile.close()
readf.close()
indf.close()


In [ ]:
# !tail -10 indexs/globmap22
# !head indexsf/globmap
# print(map_docidattr)


{'1': 'AccessibleComputing:2:1:1:5:1:1:11:', '2': 'Anarchism:2:1:12:3566:99:1:3681:', '3': 'AfghanistanHistory:2:1:1:6:1:1:12:', '4': 'AfghanistanGeography:2:1:1:6:1:1:12:', '5': 'AfghanistanPeople:2:1:1:6:1:1:12:', '6': 'AfghanistanCommunications:2:1:1:6:1:1:12:', '7': 'AfghanistanTransportations:2:1:1:7:1:1:13:', '8': 'AfghanistanMilitary:2:1:1:7:1:1:13:', '9': 'AfghanistanTransnationalIssues:2:1:1:7:1:1:13:', '10': 'AssistiveTechnology:2:1:1:5:1:1:11:', '11': 'AmoeboidTaxa:2:1:1:5:1:1:11:', '12': 'Autism:2:58:11:4830:47:1:4949:', '13': 'AlbaniaHistory:2:1:1:6:1:1:12:', '14': 'AlbaniaPeople:2:1:1:6:1:1:12:', '15': 'AsWeMayThink:2:1:1:5:1:1:11:', '16': 'AlbaniaGovernment:2:1:1:6:1:1:12:', '17': 'AlbaniaEconomy:2:1:1:6:1:1:12:', '18': 'Albedo:2:1:10:1950:24:67:2054:', '19': 'AfroAsiaticLanguages:2:1:1:6:1:1:12:', '20': 'ArtificalLanguages:2:1:1:9:1:1:15:', '21': 'AbacuS:2:1:1:5:1:1:11:', '22': 'AbalonE:2:1:1:5:1:1:11:', '23': 'AbbadideS:2:1:1:6:1:1:12:', '24': 'AbbesS:2:1:1:5:1:1:11:',

In [ ]:
######################################################
#############          search part
############BElIEVE IN GOD AND WORK HARD------------------ Everything else will fall in place
############# created by shanu gandhi on 30 -08- 2020
######   sec ind format
# chunk_0:000
# chunk_1:incab
# chunk_2:variationsefncatherin
# chunk_3:efnnamesurfac
# chunk_4:postextorgwork550
#
#
import nltk
import re
nltk.download('punkt',quiet=True)
nltk.download('stopwords',quiet=True)
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem import SnowballStemmer 
from nltk.tokenize import word_tokenize 
import math
import time
import sys
import nltk
from nltk.stem import SnowballStemmer 
from nltk.corpus import stopwords
custset={" ","a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could","redirect","couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "wiktionary", "cite","reflist"}
custset=custset.union(set(stopwords.words('english')))
ps =nltk.stem.SnowballStemmer('english')

indpath = "indexsf/indmerged"
# querystr = "3, t:World Cup i:2019 c:Cricket"

##################
# read index file
count =-1
# mapreaded={}
start=time.time();

reslis=[]
map_docidattr={}
pickle_in = open("indexsf/dict.pickle","rb")
map_docidattr= pickle.load(pickle_in)
# def readdocidtitle():
#   f1=open("indexsf/globmap","r")
#   # line= 58576=Phosphor:2:1:6:2622:38:2:2671:
#   while True:
#     line=f1.readline()
#     if not line:
#       break
#     line=line.strip()
#     did=line.split("=")[0]
#     attr=line.split("=")[1]
#     map_docidattr[did]=attr
    

# print(len(map_docidattr))
# 't1-i3-c1-e3-b65-d38425':
def parseandsort(k, curdfull):
  curlis=curdfull.split("=")
  curdoc=curlis[1].strip()
  # print("IN p&S",curdoc)
  ll=curdoc.split(":")
  # print(str(ll))
  ps=[]
  for i in ll:
    ilis=i.split("-")  ### [t1 i3 c1 e3 b65 d38425]
    cs=0
    for j in ilis: ##j=b65
      if j!="" and j!=" " and len(j)> 1 and j[0]!='d':
        # print(j,":",j[1:])
        ##### Assign weight to title =5X , infobox=3X, Categ= 3X, ext link=1.5X, Refs=1.5X, Body=1X
        if j[0]=='t':
          cs+=((int(j[1:]))*5)
        elif j[0]=='i':
          cs+=((int(j[1:]))*3)
        elif j[0]=='c':
          cs+=((int(j[1:]))*3)
        elif j[0]=='e':
          cs+=((int(j[1:]))*1.5)
        elif j[0]=='r':
          cs+=((int(j[1:]))*1.5)
        else:
          cs+=(int(j[1:]))
    docid=ilis[-1][1:]
    if cs >0 :
      ps.append([cs, docid]) 
  # print(str(ps))
  ps.sort(key=lambda x: x[0],reverse =True)
  # print(str(ps))
  for i in range( min(int(k),len(ps)) ):
    # print(i)
    # print((ps[i][1]))
    # print( map_docidattr[(ps[i][1])])
    print( map_docidattr[(ps[i][1])].split(":")[0])





def parseandsortm(topK,reslis):
  lisg=[]
  setdocs= set()
  for i1 in reslis:
    curdfull=i1
    # print(str(i1))
    curlis=curdfull.split("=")
    curdoc=curlis[1].strip()
    # print("IN p&S",curdoc)
    ll=curdoc.split(":")
    # print(str(ll))
    plsize=len(ll)
    # print("pls",plsize)
    # ps=[]
    ps={}
    for i in ll:

      ilis=i.split("-")  ### [t1 i3 c1 e3 b65 d38425]
      cs=0
      for j in ilis: ##j=b65
        if j!="" and j!=" " and len(j)> 1 and j[0]!='d':
            cs+=(int(j[1:]))

      docid=ilis[-1][1:]
      if cs >0 :
        tw=map_docidattr[docid].split(":")[-2]  ####### m1[58576] =   Phosphor:2:1:6:2622:38:2:2671:\n
        tf=math.log(1+(int(cs)/int(tw)))
        idf=math.log(len(map_docidattr)/int(plsize))
        tfidf=round(tf*idf, 10)
        ps[docid]=tfidf
        setdocs.add(docid)
    lisg.append(ps)
  lisf=[]
  for i in setdocs:
    cur=0
    ct=0
    for j in lisg:
      if i in j:
        cur+=j[i]
        ct=ct+1
    lisf.append([i,ct,cur])
  lisf = sorted(lisf, key = lambda x: (x[1], x[2]))
  lisf.reverse()
  for i in range( min(int(topK),len(lisf)) ):
    print( map_docidattr[(lisf[i][0])].split(":")[0])


def parseandsortfield(topK,reslis):
  # print(topK)
  lisg=[]
  setdocs= set()
  
  for i1 in reslis:
    # print(i)
    curtag=i1[0:1]
    curdfull=i1[2:]
#    print(str(curdfull))
    curlis=curdfull.split("=")
    curdoc=curlis[1].strip()
    # print("IN p&S",curdoc)
    ll=curdoc.split(":")
    # # print(str(ll))
    plsize=len(ll)
    # # print("pls",plsize)
    # ps=[]
    ps={}
    for i in ll:
      ilis=i.split("-")  ### [t1 i3 c1 e3 b65 d38425]
      cs=0
      for j in ilis: ##j=b65 ####adding all value here add only tag value req
        if j!="" and j!=" " and len(j)> 1 and j[0]!='d'and j[0]==curtag:
            cs+=(int(j[1:]))

      docid=ilis[-1][1:]
      if cs >0 :
        tw=map_docidattr[docid].split(":")[-2]  ####### m1[58576] =   Phosphor:2:1:6:2622:38:2:2671:\n
        tf=math.log(1+(int(cs)/int(tw)))
        idf=math.log(len(map_docidattr)/int(plsize))
        tfidf=round(tf*idf, 10)
        ps[docid]=tfidf
        setdocs.add(docid)
    lisg.append(ps)
  # for i in lisg:
  #   print(i)
  lisf=[]
  for i in setdocs:
    cur=0
    ct=0
    for j in lisg:
      if i in j:
        cur+=j[i]
        ct=ct+1
    lisf.append([i,ct,cur])
  lisf = sorted(lisf, key = lambda x: (x[1], x[2]))
  lisf.reverse()
  for i in range( min(int(topK),len(lisf)) ):
    print( map_docidattr[(lisf[i][0])].split(":")[0])




def searchnormal(query):
  global reslis
  stmq=ps.stem(query)
  # print("normal:",stmq)
  # sind=open("indexs/chunksf/secndind","r")
  sind=open("indexsf/chunksdir/secndind","r")
  sline =sind.readlines()
  ftosearch=""
  for i in range(len(sline)-1):
    curfw=sline[i].split(":")[1].strip()
    nextfw=sline[i+1].split(":")[1].strip()
    if curfw <= stmq and nextfw > stmq:
      ftosearch=sline[i].split(":")[0].strip()
      # print("found")
      break
  if ftosearch=="":
    ftosearch=sline[-1].split(":")[0].strip()
  # print("chunk to searc", ftosearch)
  fc=open("indexsf/chunksdir/"+ftosearch,"r")
  while True:
    line=fc.readline()
    if not line or line[0]>stmq[0]:
      break
    if line.startswith(stmq+"="):
      # print(line)
      reslis.append(line)
      break
  print("")

def searchcateg(query):
  global reslis
  tag=query[0:1]
  query=query[2:]
  lisq=query.split()
  lisq.insert(0,tag)
  # print(lisq)
  curt=""
  sind=open("indexsf/chunksdir/secndind","r")
  sline =sind.readlines()
  for j in lisq:
    if j=='i' or j=='t'or j=='c'or j=='r'or j=='b'or j=='e':
      curt=j
      continue;
    stmq=ps.stem(j)
    # print(curt, ":",stmq)
    ftosearch=""

    for i in range(len(sline)-1):
      curfw=sline[i].split(":")[1].strip()
      nextfw=sline[i+1].split(":")[1].strip()
      if curfw <= stmq and nextfw > stmq:
        ftosearch=sline[i].split(":")[0].strip()
        # print("found")
        break
    
    if ftosearch=="":
      # print("line",sline)
      ftosearch=sline[-1].split(":")[0].strip()
    # print("chunk to searc", ftosearch)
    fc=open("indexsf/chunksdir/"+ftosearch,"r")
    while True:
      line=fc.readline()
      if not line or line[0]>stmq[0]:
        break
      if line.startswith(stmq+"="):
        # print(line)
        reslis.append(curt+":"+line)
        break
    # print("")
  return 




#####################   main execution

readdocidtitle()
# querystr = "10, Narendra"
# querystr = "10, Shanu"
querystr="3, t:World Cup i:2019 c:Cricket"
q=querystr
q=q.lower()
qlis=q.split(",")
topK=qlis[0]
# print(topK)
q=qlis[1]
if q[2]!=":":
  # print("Norm str")
  global reslis
  reslis=[]
  lis=q.split(' ')
  for i in lis:
    if i!="":
      searchnormal(i)
  # print(reslis)
  if len(reslis)==0:
    print("No Document Found")
    pass
  elif len(reslis)==1:
    parseandsort(topK,reslis[0])
  else:
    print("MULTIPLE WORDS")
    parseandsortm(topK,reslis)
else:
  # print("Categ query")
  j=0
  cs=""
  q=q[1:]
  # print(q)
  lisq=q.split(":")
  # print(lisq)
  flis=[]
  cur=""
  flis.append(lisq[0]+":"+lisq[1][:-2])
  for i in range(len(lisq)-1):
    if i==0:
      continue
    if i==len(lisq)-2:
      cur=cur+lisq[i][-1]+":"+lisq[i+1]
    else:
      cur=cur+lisq[i][-1]+":"+lisq[i+1][:-1]
    cur=cur.strip()
    flis.append(cur)
    cur=""
  # print(flis)
  reslis=[]
  for i in flis:
    searchcateg(i)
  # print(len(reslis))
  # for i in reslis:
  #   print(i)
  if len(reslis)==0:
    print("No Document Found")
    pass
  else:
    # print("MULTIPLE DOCLIST")
    parseandsortfield(topK,reslis)

Cricket World Cup
Rugby World Cup
1966 FIFA World Cup


In [ ]:
# ###############check if ind is sorted
# # fp=open("indexs/indmerged")
# # while True:
# ftmp=open("indexs/chunksf/chunk_3")
# j=0
# while True:
#   j=j+1
#   if j==1000:
#     break
#   line=ftmp.readline()
#   if not line: 
#     break
#   # if line.startswith("w"):
#     # print(line)
#   print(line)


In [ ]:
# # !grep 'world' "indexs/chunksf/chunk_3"
fpt=open("indexsf/indmerged","r")
while True:
  line=fpt.readline()
  if line.startswith("shanu="):
    print(line)
    break

shanu=i1-d4509:b1-d6819:b1-d56489:



In [ ]:
!head -10 indexs/

000=b1d100b1d169i1d177b1d389b1d395b1d404i1d581i1d790b1d926b2d998b1d1205i2d1774i1d1870b1d1880i1d1925i1d1951i3d1953b1d2095i1d2096b1d2098b1d2112i3d2119i1d2243i2d2291i1d2453b1d2574i13d2848b1d2959i1d3051b1d3149b1d3167b1d3255b1d3282b1d3347b3d3349i1d3543i1d3751i2d3783b2d3854e2d3942i1d4111b1d4123i2d4151b1d4645b1d4743b3d4745b3d4925i2d5068b6d5165b1d5370i1d5702i1d5709b1d5720i1d5777b1d5778i1d5791i1d5797i2d5890i1d5896i1d5922b1d6055i1d6555i2d6637i2d6734i1d6918b1d6994b1d7046b1d7063b2d7185i2d7222b1d7529i1d7602b1d7692b4d7827i2d7856e1d8027b5d8031i1d8483i1b5d8520i1d8589i5d8745b1d8813b1d9178i2d9244i1d9286b1d9449e1d9644i1d10400i14d10814b1d11287i1d11391b1d11403b1d11425b9d11545i2d12009i1d12084e1d12131b1d12219i1b1d12249i1d12304i1d12319b1d12332i1d12355b1d12356b1d12396b1d12400b1d12506b1d12910b1d12921b1d13043i2d13131i1d13286i2d13306i1d13315b1d13580i1d13750b1d13815i1d13820b1d13854i1d13868b1d13888i1d13979b1d13982b3d14115b2d14224b1d14233b1d14377i1d14421i2d14497i1d14569b1d14604b1d14810i1d14921b1d15252i2d15268b1d1527